### 参数管理

- 访问参数，用于调试、诊断和可视化
- 参数初始化
- 在不同模型组件间共享参数

我们首先关注具有单隐藏层的多层感知机

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.3049],
        [-0.3256]], grad_fn=<AddmmBackward0>)

**参数访问**

- **当我们通过`Sequential`类定义模型时，我们可以通过索引来访问模型的任意层。模型就像一个列表一样，每层的参数都在其属性中**

In [2]:
print(net[2].state_dict())  # state_dict()返回一个从参数名称映射到参数Tensor的字典，也是一个OrderedDict

OrderedDict([('weight', tensor([[-0.0445, -0.2072, -0.0634,  0.1603,  0.1615, -0.3044, -0.3237,  0.1320]])), ('bias', tensor([-0.1755]))])


**目标参数**

- **注意，每个参数都表示为参数类的一个实例，进一步访问该参数的值**

In [3]:
print(type(net[2].bias))  # Parameter是一个可以优化的参数类型
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1755], requires_grad=True)
tensor([-0.1755])


- **参数是复合的对象，包含值、梯度和额外信息。这就是我们需要显示参数值的原因**

In [4]:
net[2].weight.grad == None  # 还没有计算梯度

True

**一次性访问所有参数**

In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
# `net[0].named_parameters()`返回一个生成器，该生成器会产生子模块（在这种情况下是第一个子模块）的参数及其名称。每个参数都是一个张量，而名称是一个字符串。
# `name`是参数的名称，`param`是参数的张量。
# `[(name, param.shape) for name, param in net[1].named_parameters()]`是一个列表推导式，它遍历`net[0]`子模块的参数，并为每个参数生成一个元组，元组包含参数的名称和形状。
# `print(*[(name, param.shape) for name, param in net[0].named_parameters()])`使用`print`函数打印了这个列表中的每个元组。`*`符号用于展开列表，使其成为`print`函数的多个参数，从而以易读的方式打印出参数名称和形状。

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [6]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.1107,  0.2264, -0.3752,  0.4851],
                      [-0.4663,  0.0569, -0.2543,  0.4361],
                      [ 0.4925,  0.2992, -0.1242, -0.0355],
                      [ 0.1270, -0.3355,  0.2761,  0.4600],
                      [ 0.2531,  0.0652, -0.4667,  0.1429],
                      [ 0.3827, -0.1661,  0.4897,  0.0589],
                      [ 0.3709, -0.2839, -0.4068,  0.3296],
                      [-0.4619,  0.1785,  0.4826, -0.3091]])),
             ('0.bias',
              tensor([-0.3047, -0.1058, -0.3604,  0.4079, -0.1738,  0.3326, -0.1203,  0.3495])),
             ('2.weight',
              tensor([[-0.0445, -0.2072, -0.0634,  0.1603,  0.1615, -0.3044, -0.3237,  0.1320]])),
             ('2.bias', tensor([-0.1755]))])

In [50]:
net.state_dict()["2.bias"].data

tensor([-0.3529])

**从嵌套块收集参数**

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())


def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f"block{i}", block1())  # add_module是nn.Sequential类的方法，可以起名字
    return net


rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.1491],
        [0.1491]], grad_fn=<AddmmBackward0>)

**我们已经设计了网络，让我们看看它是如何组织的**

In [52]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


**指定block2打印**

In [8]:
print(rgnet[0][2])

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
)


In [9]:
# 指定block2的第0层的权重参数
rgnet[0][2][0].weight.data

tensor([[ 0.0980, -0.4231,  0.2057,  0.1557],
        [-0.4521, -0.2525, -0.4275,  0.4382],
        [-0.1174,  0.1434, -0.0364,  0.1481],
        [-0.1590,  0.0311,  0.3790, -0.3063],
        [ 0.2108,  0.2156, -0.0316,  0.2121],
        [ 0.1660,  0.3180,  0.3667, -0.2781],
        [-0.3435,  0.4025,  0.0153, -0.1299],
        [ 0.1497,  0.4966, -0.4129, -0.4125]])

**内置初始化**

- **默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，这个范围是根据输入维度和输出维度计算出的。**

- **PyTorch 的 `nn.init` 模块提供了多种内置初始化方法**

In [10]:
def init_normal(m):  # m is a layer
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)


net.apply(init_normal)  # 对所有层使用init_normal函数初始化
net[0].weight.data[0], net[0].bias.data[0]  # 查看第一层的第一行的参数

(tensor([ 0.0005,  0.0114,  0.0103, -0.0143]), tensor(0.))

In [56]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)


net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

**对某些块应用不同的初始化方法**

In [57]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)


def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)


net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5401,  0.3312, -0.6944, -0.2573])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


**自定义初始化**

In [14]:
net, [(name, param.shape) for name, param in net[2].named_parameters()][0]

(Sequential(
   (0): Linear(in_features=4, out_features=8, bias=True)
   (1): ReLU()
   (2): Linear(in_features=8, out_features=1, bias=True)
 ),
 ('weight', torch.Size([1, 8])))

- **有时，深度学习框架没有提供我们需要的初始化方法。在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：**

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


In [18]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)

        # 使用逐元素的比较运算 `m.weight.data.abs() >= 5` 来创建一个布尔张量，表示权重绝对值大于等于 5 的位置
        # 通过执行 `m.weight.data *= m.weight.data.abs() >= 5`，将权重中绝对值大于等于 5 的元素置零，从而达到一个稀疏化的效果
        m.weight.data *= m.weight.data.abs() >= 5


net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  6.1215, -0.0000,  0.0000],
        [-6.7285, -6.0349, -9.1026, -0.0000]], grad_fn=<SliceBackward0>)

**我们始终可以直接设置参数**

In [16]:
net[0].weight.data[:] += 1  # 原地改变
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  9.4046,  1.0000,  1.0000])

-----------------------

### **参数绑定**

- **当参数绑定时，梯度会发生什么情况？**

  - 由于模型参数包含梯度，**因此在反向传播期间，绑定的参数的梯度将累积所有梯度**。

In [69]:
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1)
)
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
